<a href="https://colab.research.google.com/github/YesidCastelblanco/Extraccion_y_almacenamiento_de_datos/blob/main/WEB_SCRAPING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


# ***Práctica Web Scraping***


---



In [53]:
# Instalamos las librerias que vamos a utilizar para el ejercicio de web scraping
!pip install beautifulsoup4
!pip install lxml
!pip install requests

In [54]:
# Importamos las librerias
import requests
from bs4 import BeautifulSoup
import pandas as pd
from IPython.display import display

In [56]:
# URL base de la página de prueba de web scraping
base_url = 'https://books.toscrape.com/catalogue/page-{}.html'

In [57]:
# Asignamos un control de paginación
page_number = 1

# Listas para almacenar los datos
titles = []
prices = []
availability = []
ratings = []

while True:
    # Crear la URL de la página actual
    url = base_url.format(page_number)

    # Hacer una solicitud a la página
    response = requests.get(url)

    # Verificar el código de estado
    if response.status_code != 200:
        print(f"Error al acceder a la página: {response.status_code}")
        break

    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    books = soup.find_all('article', class_='product_pod')

    # Si no hay libros en la página, salir del bucle
    if not books:
        break

    # Procesar cada libro extraído
    for book in books:
        title = book.find('h3').find('a')['title']
        price = book.find('p', class_='price_color').get_text(strip=True)
        availability_text = book.find('p', class_='instock availability').get_text(strip=True)
        rating_class = book.find('p')['class']
        rating = rating_class[1] if len(rating_class) > 1 else 'No rating'

        titles.append(title)
        prices.append(price)
        availability.append(availability_text)
        ratings.append(str(rating))

    # Aumentamos el número de página para la próxima iteración
    page_number += 1

# Crear el DataFrame
df = pd.DataFrame({
    'Título': titles,
    'Precio': prices,
    'Disponibilidad': availability,
    'Rating': ratings
})

# Cambiar opciones de visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Mostrar el DataFrame
display(df)

# Mostrar el total de libros extraídos
print(f'Total de libros extraídos: {len(df)}')


Error al acceder a la página: 404


,Título,Precio,Disponibilidad,Rating
0,A Light in the Attic,£51.77,In stock,Three
1,Tipping the Velvet,£53.74,In stock,One
2,Soumission,£50.10,In stock,One
3,Sharp Objects,£47.82,In stock,Four
4,Sapiens: A Brief History of Humankind,£54.23,In stock,Five
...,...,...,...,...
995,Alice in Wonderland (Alice's Adventures in Won...,£55.53,In stock,One
996,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",£57.06,In stock,Four
997,A Spy's Devotion (The Regency Spies of London #1),£16.97,In stock,Five
998,1st to Die (Women's Murder Club #1),£53.98,In stock,One


Total de libros extraídos: 1000


# Buscamos el libro con el rating más alto

In [61]:
# Pasamos los rating a valores numéricos
rating_map = {
    'Five': 5,
    'Four': 4,
    'Three': 3,
    'Two': 2,
    'One': 1,
    'No rating': 0
}

# Creamos una nueva columna en el DataFrame con los ratings numéricos
df['Rating Numeric'] = df['Rating'].map(rating_map)

# Encontramos el índice del libro con el rating más alto
highest_rating_index = df['Rating Numeric'].idxmax()

# Obtenemos el libro con el rating más alto
highest_rated_book = df.iloc[highest_rating_index]

# Mostrar el libro con la calificación más alta
print("El libro con la calificación más alta es:")
print(highest_rated_book[['Título', 'Precio', 'Disponibilidad', 'Rating']])

El libro con la calificación más alta es:
Título            Sapiens: A Brief History of Humankind
Precio                                           £54.23
Disponibilidad                                 In stock
Rating                                             Five
Name: 4, dtype: object


# Buscamos el libro mas costoso

In [60]:
# Convertimos los precios a valores numéricos (float), igualmente, quitamos el simbolo de la moneda
df['Precio Numeric'] = df['Precio'].replace({'£': '', ' ': ''}, regex=True).astype(float)

# Buscamos el índice del libro con el precio más alto
highest_price_index = df['Precio Numeric'].idxmax()

# Obtener el libro con el precio más alto
highest_priced_book = df.iloc[highest_price_index]

# Imprimimos el libro con el precio más alto
print("\nEl libro más costoso es:")
print(highest_priced_book[['Título', 'Precio', 'Disponibilidad', 'Rating']])



El libro más costoso es:
Título            The Perfect Play (Play by Play #1)
Precio                                        £59.99
Disponibilidad                              In stock
Rating                                         Three
Name: 648, dtype: object
